In [ ]:
from nbdev import *
%nbdev_default_export functions
#%nbdev_default_class_level 3

Cells will be exported to pct.functions,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
%nbdev_hide
%reload_ext autoreload
%autoreload 2

# Functions

> Functions that form the elements of a perceptual control node (system).


## Overview
Each function outputs the result of applying the function logic and may be a scalar or array, depending on the input. The inputs are supplied as links to another function, from where the values are read. Unless the function is a simple case, such as a 'Constant'. 

Parameters are supplied as arguments to the constructor of the class.

In [ ]:
# hide
import sys
sys.path.append("..")

In [ ]:
%nbdev_export
import numpy as np
import gym
from abc import ABC, abstractmethod
from pct.utilities import UniqueNamer
from pct.utilities import FunctionsList

In [ ]:
%nbdev_export
class BaseFunction(ABC):
    "Base class of a PCT function. This class is not used direclty by developers, but defines the functionality common to all."
    def __init__(self, name, value, links=None, new_name=True):
        self.value = value
        self.links = []
        self.handle_links(links)

                
        #print(f'size {len(UniqueNamer.getInstance().names)} {name} {name in UniqueNamer.getInstance().names}', end=" ")
        if new_name:
            self.name = UniqueNamer.getInstance().get_name(name)
        else:
            self.name = name                
        #print(self.name)
        FunctionsList.getInstance().add_function(self)
        self.decimal_places = 3
        
    @abstractmethod
    def __call__(self, verbose=False):
        if verbose :
            print(self.output_string(), end= " ")
            
        return self.value
    def handle_links(self, links):
        if links!=None:
            if isinstance(links, dict):
                if len(links)>0:
                    for key in links.keys():
                        self.links.append(FunctionsList.getInstance().get_function(links[key]))
                    return

            if isinstance(links, list):
                for link in links:
                    self.links.append(link)
                    #if isinstance(link, dic):
                    #    self.links.append(FunctionsList.getInstance().get_function(link))
                    #else:
            #else:

                #if isinstance(links, str):
                 #   self.links.append(FunctionsList.getInstance().get_function(links))
                #else:
                 #   self.links.append(links)
        
        
        
        
    def output_string(self):
        return f'{round(self.value, self.decimal_places):.{self.decimal_places}f}'
    
    def check_links(self, num):
        if len(self.links) != num:
            raise Exception(f'Incorrect number of links {len(self.links)} for function {self.name}. {num} expected.')            
        
    def set_decimal_places(self, dp):
        self.decimal_places = dp
        
    @abstractmethod    
    def summary(self, str):
        print(f'{self.name} {type(self).__name__}', end = " ")
        if len(str)>0:
            print(f'| {str}', end= " ")
        print(f'| {self.value}', end = " ")
        if len(self.links)>0:
            print('| links ', end=" ")
        for link in self.links:
            print(link.get_name(), end= " ")
        print()
        
    @abstractmethod    
    def get_config(self):
        config = {"type": type(self).__name__,
                    "name": self.name, 
                    "value": self.value}
        
        ctr=0
        links={}
        for link in self.links:
            links[ctr]=link.get_name()
            ctr+=1
        
        config['links']=links
        return config
        
    def get_name(self):
        return self.name
    
    def set_name(self, name):
        self.name=name

    def set_value(self, value):
        self.value= value
    
    def get_value(self):
        return self.value
    
    def get_indexed_value(self, index):
        return self.value[index]

    def add_link(self, linkfn):
        self.links.append(linkfn)
    
    def close(self):
        pass
    
    @classmethod
    def from_config(cls,  config):
        #print("a:",config)
        func = cls(new_name=False, **config)
        #key  = 'links'
        #if key in config:
        #    for key in config['links'].keys():
        #        func.links.append(FunctionsList.getInstance().get_function(config['links'][key]))
        
        #print("b:",func.get_config())
        return func

    def __str__(self):
        return str(self.__dict__)

In [ ]:
%nbdev_export
class Proportional(BaseFunction):
    "A proportion of the input value as defined by the gain parameter. Parameters: The gain value. Links: One."
    def __init__(self, gain=1, value=0, name="proportional", links=None, new_name=True, **cargs):
        super().__init__(name, value, links, new_name)
        self.gain = gain

    def __call__(self, verbose=False):
        input = self.links[0].get_value()
        self.value = input * self.gain
        return super().__call__(verbose)
    
    def summary(self):
        super().summary(f'gain {self.gain}')

    def get_config(self):
        config = super().get_config()
        config["gain"] = self.gain
        return config               

In [ ]:
%nbdev_export
class Variable(BaseFunction):
    "A function that returns a variable value. Parameter: The variable value. Links: None"
    def __init__(self,  value=0, name="variable", links=None, new_name=True, **cargs):
        super().__init__(name, value, links, new_name)
    
    def __call__(self, verbose=False):
        return super().__call__(verbose)
    
    def summary(self):
        super().summary("")
        
    def get_config(self):
        config = super().get_config()
        return config



In [ ]:
%nbdev_export
class GreaterThan(BaseFunction):
    "One of two supplied values is returned if the input is greater than supplied threshold.</br> Parameters: The threshold and upper and lower value. Links: One"
    def __init__(self, threshold=0, upper=1, lower=0, value=0, name="greaterthan", links=None, new_name=True, **cargs):
        super().__init__(name, value, links, new_name)
        self.threshold=threshold
        self.upper=upper
        self.lower=lower
    
    def __call__(self, verbose=False):
        input = self.links[0].get_value()
        if input >= self.threshold:
            self.value = self.upper
        else:
            self.value = self.lower
        
        return super().__call__(verbose)
    
    def summary(self):
        super().summary("")
        
    def get_config(self):
        config = super().get_config()
        return config



In [ ]:
%nbdev_export
class Subtract(BaseFunction):
    "A function that subtracts one value from another. Parameter: None. Links: Two links required to each the values to be subtracted."
    def __init__(self, value=0, name="subtract", links=None, new_name=True, **cargs):
        super().__init__(name, value, links, new_name)
    
    def __call__(self, verbose=False):
        #print("Sub ", self.links[0].get_value(),self.links[1].get_value() )
        self.value = self.links[0].get_value()-self.links[1].get_value()

        return super().__call__(verbose)

    def summary(self):
        super().summary("")

    def get_config(self):
        return super().get_config()
                        

In [ ]:
%nbdev_export
class Constant(BaseFunction):
    "A function that returns a constant value. Parameter: The constant value. Links: None"
    def __init__(self, value=0, name="constant", new_name=True, **cargs):
        super().__init__(name, value, None, new_name)
    
    def __call__(self, verbose=False):
        return super().__call__(verbose)
    
    def summary(self):
        super().summary("")

    def get_config(self):
        return super().get_config()


In [ ]:
%nbdev_export
class Integration(BaseFunction):
    "A leaky integrating function. Equivalent of a exponential smoothing function, of the amplified input. Parameter: The gain and slow values. Links: One."
    def __init__(self, gain=1, slow=2, value=0, name="integration", links=None, new_name=True, **cargs):
        super().__init__(name, value, links, new_name)
        self.gain = gain
        self.slow = slow
    
    def __call__(self, verbose=False):
        input = self.links[0].get_value()
        self.value = self.value +  ((input * self.gain) - self.value)/self.slow
        
        return super().__call__(verbose)

    def summary(self):
        super().summary(f'gain {self.gain} slow {self.slow} ')

    def get_config(self):
        config = super().get_config()
        config["gain"] = self.gain
        config["slow"] = self.slow
        return config       
        

In [ ]:
%nbdev_export
class WeightedSum(BaseFunction):
    "A function that combines a set of inputs by multiplying each by a weight and then adding them up. Parameter: The weights array. Links: Links to all the input functions."
    def __init__(self, weights=np.ones(3), value=0, name="weighted_sum", links=None, new_name=True, **cargs):
        super().__init__(name, value, links, new_name)
        self.weights = weights
    
    def __call__(self, verbose=False):
        if len(self.links) != self.weights.size:
            raise Exception(f'Number of links {len(self.links)} and weights {self.weights.size} must be the same.')
        
        inputs = np.array([link.get_value() for link in self.links])
        self.value = np.dot(inputs, self.weights)

        return super().__call__(verbose)

    def summary(self):
        super().summary("")

    def get_config(self):
        config = super().get_config()
        config["weights"] = self.weights
        return config

In [ ]:
%nbdev_export
class IndexedParameter(BaseFunction):
    "A function that returns a parameter from a linked function, indexed by number. Parameter: The index. Links: One."
    def __init__(self, index=None, value=0, name="indexed_parameter", links=None, new_name=True, **cargs):
        super().__init__(name, value, links, new_name)
        self.index = index
    
    def __call__(self, verbose=False):
        super().check_links(1)
        self.value = self.links[0].get_indexed_value(self.index)

        return super().__call__(verbose)

    def summary(self):
        super().summary(f'index {self.index}')


    def get_config(self):
        config = super().get_config()
        config["index"] = self.index
        return config

In [ ]:
%nbdev_export
class OpenAIGym(BaseFunction):
    "A function that creates an runs an environment from OpenAI Gym. Parameter: The environment name. Flag to display environment. Links: Link to the action function."
    def __init__(self, env_name=None, render=False, video_wrap=False, value=0, name="gym", links=None, new_name=True, **cargs):
        super().__init__(name, value, links, new_name)
        
        self.video_wrap = video_wrap
        self.create_env(env_name, 4000)
        self.render = render
        self.reward = 0
        self.done = False
        self.info = {}
        
    def __call__(self, verbose=False):
        super().check_links(1)
        input = self.links[0].get_value()
        if input == 1 or input == -1 or input == 0:
            obs = self.env.step(input)
        else:
            raise Exception(f'OpenAIGym: Input value of {input} is not valid, must be 1,0 or -1.')
        
        self.value = obs[0]
        
        self.reward = obs[1]
        self.done = obs[2]
        self.info = obs[3]
        
        if self.render:
            self.env.render()
            
        return super().__call__(verbose)

    def summary(self):
        super().summary("")

    def get_config(self):
        config = super().get_config()
        config["values"] = self.value
        config["reward"] = self.reward
        config["done"] = self.done
        config["info"] = self.info
        
        return config
    
    def output_string(self):
        
        if isinstance(self.value, int):
            rtn = f'{round(self.value, self.decimal_places):.{self.decimal_places}f}'
        else:
            list = [f'{round(val, self.decimal_places):.{self.decimal_places}f} ' for val in self.value]
            list.append(str(self.reward))
            list.append(" ")
            list.append(str(self.done))
            list.append(" ")
            list.append(str(self.info))
            print(list)
            
            rtn = ''.join(list)
            """
            str = ""
            for val in self.value:
                str+= f'{round(val, self.decimal_places):.{self.decimal_places}f} '

            str+= str(self.reward) + " "
            str+= self.done + " "
            str+= self.info 
        """
        return rtn

    
    def create_env(self, env_name, max_episode_steps):
        genv = gym.make(env_name)
        genv._max_episode_steps = max_episode_steps
        if self.video_wrap:
            self.env =  vid.wrap_env(genv)
        else:
            self.env = genv
            self.env.reset()
            
    def close(self):
        self.env.close()


## Creating Functions

Standard class constructor. Different ways to create a function with the standard constructor.

In [ ]:
prop = Proportional()
print(prop.get_config())
prop = Proportional("myprop", 10)
print(prop.get_config())
prop = Proportional(gain=10)
print(prop.get_config())

{'type': 'Proportional', 'name': 'proportional', 'value': 0, 'links': {}, 'gain': 1}
{'type': 'Proportional', 'name': 'proportional1', 'value': 10, 'links': {}, 'gain': 'myprop'}
{'type': 'Proportional', 'name': 'proportional2', 'value': 0, 'links': {}, 'gain': 10}


Configuration class constructor. Create the function by passing a configuration structure to the constructor.

In [ ]:
prop = Proportional(**{'name': 'myprop', 'value': 5, 'gain': 20})
print(prop.get_config())

{'type': 'Proportional', 'name': 'myprop1', 'value': 5, 'links': {}, 'gain': 20}


Configuration class method. Create the function by passing a configuration structure to a class method.

In [ ]:
config = {'name': 'myprop', 'value': -0.5, 'gain': 21}
prop = Proportional.from_config(config)
print(prop.get_config())

{'type': 'Proportional', 'name': 'myprop', 'value': -0.5, 'links': {}, 'gain': 21}


In [ ]:
prop = Proportional()
print(prop.get_config())
prop1 = Proportional.from_config(prop.get_config())
print(prop1.get_config())
assert prop.get_config() == prop1.get_config()

{'type': 'Proportional', 'name': 'proportional3', 'value': 0, 'links': {}, 'gain': 1}
{'type': 'Proportional', 'name': 'proportional3', 'value': 0, 'links': {}, 'gain': 1}


An example showing creating a WeightedSum function.

In [ ]:
wts=np.ones(3)
ws = WeightedSum(weights=wts)
ws.add_link(Constant(10))
ws.add_link(Constant(5))
ws.add_link(Constant(20))
assert ws() == 35
ws.get_config()

{'type': 'WeightedSum',
 'name': 'weighted_sum',
 'value': 35.0,
 'links': {0: 'constant2', 1: 'constant3', 2: 'constant4'},
 'weights': array([1., 1., 1.])}

In [ ]:
ws1 = WeightedSum.from_config(ws.get_config())
ws1.get_config()

{'type': 'WeightedSum',
 'name': 'weighted_sum',
 'value': 35.0,
 'links': {0: 'constant2', 1: 'constant3', 2: 'constant4'},
 'weights': array([1., 1., 1.])}

## Viewing Functions

View the details of the function with the "summary", which prints the name, type, parameters, value and links (if any).

In [ ]:
prop.summary()

proportional3 Proportional | gain 1 | 0 


As already seen the function details can be seen by retrieving the configuration.

In [ ]:
print(prop.get_config())

{'type': 'Proportional', 'name': 'proportional3', 'value': 0, 'links': {}, 'gain': 1}


Or you can print the function.

In [ ]:
print(prop)

{'value': 0, 'links': [], 'name': 'proportional3', 'decimal_places': 3, 'gain': 1}


Set the decimal places for output display.

In [ ]:
print(prop.output_string())
prop.set_decimal_places(2)
print(prop.output_string())

0.000
0.00


# Setting Links
The next cell shows how a link is added to one function from another. In this case from an Integration function to a Constant function. So, whenever "integrator" runs it will get its input from "cons". 

In [ ]:
integrator = Integration(3, 10)
cons = Constant(5)
integrator.add_link(cons)
integrator.summary()

integration1 Integration | gain 3 slow 10  | 0 | links  constant5 


In [ ]:
#print(UniqueNamer.getInstance().names)
UniqueNamer.getInstance().clear()
integ = Integration(**{'name': 'myinteg', 'value': 1, 'gain': 20, 'slow': 100})
prop = Proportional(5, name="myprop")
integ.add_link(prop)
print(integ.get_config())
assert integ.get_config() == {'type': 'Integration', 'name': 'myinteg', 'value': 1, 'links': {0: 'myprop'}, 'gain': 20, 'slow': 100}

{'type': 'Integration', 'name': 'myinteg', 'value': 1, 'links': {0: 'myprop'}, 'gain': 20, 'slow': 100}


## Running a Fucntion
A function can simply be run by calling it, without any parameters. It will use whatever input was set by the links. It returns the result of the function. In this example it will be 5 \* 3 / 10, that is, input \* gain / slow.

In [ ]:
output = integrator()
print(output)
assert output == 1.5

1.5


## Array Input
The input to a function can be an array rather than a single value.

In [ ]:
integrator.set_value(np.array([1, 2, 4, 3]))
output = integrator()
print(output)
np.testing.assert_array_equal(output, [2.4, 3.3, 5.1, 4.2])

[2.4 3.3 5.1 4.2]


In [ ]:
print(integrator.get_config())

{'type': 'Integration', 'name': 'integration1', 'value': array([2.4, 3.3, 5.1, 4.2]), 'links': {0: 'constant5'}, 'gain': 3, 'slow': 10}


## Examples

### Configuration
Create a function from the configuration of another.

In [ ]:
#print(integrator.get_config())
integrator = Integration(3, 10)
cons = Constant(5)
integrator.add_link(cons)

inte = Integration.from_config(integrator.get_config())
print(inte())
print(inte.get_config())
target = {'type': 'Integration', 'name': 'integration', 'value': 1.5, 'links': {0: 'constant'}, 'gain': 3, 'slow': 10}
print(target)
assert inte.get_config() == target

1.5
{'type': 'Integration', 'name': 'integration', 'value': 1.5, 'links': {0: 'constant'}, 'gain': 3, 'slow': 10}
{'type': 'Integration', 'name': 'integration', 'value': 1.5, 'links': {0: 'constant'}, 'gain': 3, 'slow': 10}


### OpenAI Gym
An example showing how to use an OpenAI Gym function. And how to have another function which accesses one of the values of the gym environment. 

In [ ]:
import os
render=False
if os.name=='nt':
    render=True
acrobot = OpenAIGym("Acrobot-v1", render=render)
acrobot.add_link(Constant(1))
acrobot.get_config()

{'type': 'OpenAIGym',
 'name': 'gym',
 'value': 0,
 'links': {0: 'constant1'},
 'values': 0,
 'reward': 0,
 'done': False,
 'info': {}}

In [ ]:
acrobot()
print(acrobot.reward)
print(getattr(acrobot, "reward"))

-1.0
-1.0


In [ ]:
acrobot.output_string()

['0.999 ', '-0.043 ', '0.998 ', '-0.060 ', '0.099 ', '-0.024 ']
['0.999 ', '-0.043 ', '0.998 ', '-0.060 ', '0.099 ', '-0.024 ', '-1.0', 'False', '{}']


'0.999 -0.043 0.998 -0.060 0.099 -0.024 -1.0False{}'

In [ ]:
acrobot.value[0]

0.9990962760577714

In [ ]:
cos_angle1 = IndexedParameter(0, name="cos_angle1")
cos_angle1.add_link(acrobot)
print(cos_angle1)
print(cos_angle1.get_config())
cos_angle1()

{'value': 0, 'links': [<__main__.OpenAIGym object at 0x000001329F5B99C8>], 'name': 'cos_angle1', 'decimal_places': 3, 'index': 0}
{'type': 'IndexedParameter', 'name': 'cos_angle1', 'value': 0, 'links': {0: 'gym'}, 'index': 0}


0.9990962760577714

In [ ]:
acrobot.close()

In [ ]:
#hide
from nbdev import *
notebook2script()

Converted 00_examples.ipynb.
Converted 01_utilities.ipynb.
Converted 02_functions.ipynb.
Converted 03_nodes.ipynb.
Converted 04_hierarchy.ipynb.
Converted index.ipynb.
